In [1]:
import os
from langchain.llms import LlamaCpp
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import LlamaCppEmbeddings, OpenAIEmbeddings
from langchain.document_loaders import WebBaseLoader
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://news.ltn.com.tw/news/world/breakingnews/4382800")
data = loader.load()

In [2]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 10,length_function = len)
all_splits = text_splitter.split_documents(data)

In [7]:
# LlamaCppEmbeddings 目前有BUG造成運行速度緩慢，先用OpenAIEmbeddings代替 https://github.com/langchain-ai/langchain/pull/5066
model_path = "/home/sung/llm/chinese-alpaca-2-7b/gml-model-q4_0.bin"
# vectorstore = Chroma.from_documents(documents=all_splits, embedding=LlamaCppEmbeddings(model_path=model_path))

In [6]:
os.environ['OPENAI_API_KEY'] = ''
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [8]:
llama = LlamaCpp(
            model_path=model_path,
            n_gpu_layers=40,
            n_batch=8,
            verbose=True,
            n_ctx = 2048
        )

llama.cpp: loading model from /home/sung/llm/chinese-alpaca-2-7b/gml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 55296
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 5504
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 1.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: using CUDA for GPU acceleration
llama_model_load_internal: mem required  =  423.59 MB (+ 1024.00 MB

In [11]:

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever())
question = "LK-99 為什麼只能提供理論上的支持"
qa_chain({"query": question})

{'query': 'LK-99 為什麼只能提供理論上的支持',
 'result': '根據報導，LK-99的合成相對困難，因為只有很小一部分晶體能夠將銅原子放置在正確的位置，形成超導通道。這意味著LK-99的合成需要巧妙且奇蹟般的排列，並且只有在銅原子滲透到晶格中不太可能的位置時才能實現。因此，LK-99的合成和重現性相對困難，目前只能在理論上提供支持。'}

In [10]:

qa_chain = RetrievalQA.from_chain_type(llama,retriever=vectorstore.as_retriever())
question = "LK-99 為什麼只能提供理論上的支持"
qa_chain({"query": question})


llama_print_timings:        load time =   294.75 ms
llama_print_timings:      sample time =   302.41 ms /   235 runs   (    1.29 ms per token,   777.09 tokens per second)
llama_print_timings: prompt eval time = 47905.64 ms /  1467 tokens (   32.66 ms per token,    30.62 tokens per second)
llama_print_timings:        eval time =  6704.57 ms /   234 runs   (   28.65 ms per token,    34.90 tokens per second)
llama_print_timings:       total time = 56917.57 ms


{'query': 'LK-99 為什麼只能提供理論上的支持',
 'result': ' \n首先，LK-99是一種新的材料結構，其基於銅磷灰石（Lead phosphate）的超導體材質。它是由南韓科學家開發出來的新型態超導體材料，並且在過去幾個月中引起了全球科技圈的關注和討論。\n然而，雖然有學者宣稱成功合成了室溫常壓下的LK-99超導體，但目前尚未有其他研究團體能夠證實其實際存在性或進行到實驗階段。根據南韓科學家所發表的論文，他們提出了一個新的材料結構並使用模擬方法來推測和推算該材料的特性。然而，由於實際合成LK-99超導體仍需完成其他步驟，因此目前僅能提供理論上的支持而不是實證的支持。\n所以，雖然有學者宣稱成功合成了室溫常壓下的LK-99超導體，但目前尚未有其他研究團體能夠證實其實際存在性或進行到實驗階段。'}

In [14]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

template =  """
使用以下上下文來回答最後的問題。
如果你不知道答案，就說你不知道，不要試圖編造答案。
最多使用三個句子，並儘可能保持答案簡潔。
{context}
問題：{question}
有用的答案：
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

qa_chain = RetrievalQA.from_chain_type(
    llama,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    return_source_documents=True
)
result = qa_chain({"query": question})
result["result"]

Llama.generate: prefix-match hit

llama_print_timings:        load time =   294.75 ms
llama_print_timings:      sample time =   172.72 ms /   150 runs   (    1.15 ms per token,   868.45 tokens per second)
llama_print_timings: prompt eval time = 45715.47 ms /  1421 tokens (   32.17 ms per token,    31.08 tokens per second)
llama_print_timings:        eval time =  4022.69 ms /   149 runs   (   27.00 ms per token,    37.04 tokens per second)
llama_print_timings:       total time = 51252.23 ms


'這篇文章中提到，若要合成出真正的「室溫超導體 LK-99」，還需要找到一個特定的晶格排列。由於只有少數晶格可以恰當地將銅放置在正確的位置，因此LBNL的研究員認為該材料很難合成重現。\n\n這說明了雖然研究者成功推測出了室溫超導體 LK-99 理論上可能存在的情況，但實際上要找到一個確切的晶格排列還是需要進一步實驗和探究的。因此，LBNL的研究員認為該材料仍然需要更多的科學證明才能被廣泛應用於各種領域中。'

In [13]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    return_source_documents=True
)
result = qa_chain({"query": question})
result["result"]

'LK-99只能提供理論上的支持，因為它的合成和排列需要非常特殊的條件和位置，並且只有很小一部分晶體能夠實現這種排列。這使得LK-99的合成和重現變得非常困難。\n謝謝你的詢問！'